In [2]:
include("init.jl")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %=========>                               ]  20.3 %]  40.5 %               ]  62.2 %Fetching: [================================>        ]  79.0 %>]  99.2 % Installed CodecZlib ─ v0.5.1
 Installed IniFile ─── v0.5.0
 Installed StatsBase ─ v0.26.0
 Installed HTTP ────── v0.7.1
 Installed FileIO ──── v1.0.4
 Installed Colors ──── v0.9.5
  Building CodecZlib → `~/.julia/packages/CodecZlib/DAjXH/deps/build.log`


┌ Info: Precompiling HTTP [cd3eb016-35fb-5094-929b-558a96fad6f3]
└ @ Base loading.jl:1189


In [3]:
import HTTP, Plots, LightGraphs, JSON
const lg = LightGraphs

┌ Info: Recompiling stale cache file /home/olie/.julia/compiled/v1.0/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /home/olie/.julia/compiled/v1.0/LightGraphs/Xm08G.ji for LightGraphs [093fc24a-ae57-5d10-9952-331d41423f4d]
└ @ Base loading.jl:1187


LightGraphs

In [4]:
r = HTTP.request("GET", "http://httpbin.org/ip"; verbose=3)
println(r.status)
println(String(r.body))

DEBUG: 2018-11-29T14:03:25.528 436f ➡️  "GET /ip HTTP/1.1\r\n" (write)
DEBUG: 2018-11-29T14:03:25.545 436f ➡️  "Host: httpbin.org\r\n" (write)
DEBUG: 2018-11-29T14:03:25.545 436f ➡️  "Content-Length: 0\r\n" (write)
DEBUG: 2018-11-29T14:03:25.545 436f ➡️  "\r\n" (write)
DEBUG: 2018-11-29T14:03:26.056 436f ⬅️  "HTTP/1.1 200 OK\r\n" (readavailable)
DEBUG:                                 "Connection: keep-alive\r\n"
DEBUG:                                 "Server: gunicorn/19.9.0\r\n"
DEBUG:                                 "Date: Thu, 29 Nov 2018 14:03:25 GMT\r\n"
DEBUG:                                 "Content-Type: application/json\r\n"
DEBUG:                                 "Content-Length: 31\r\n"
DEBUG:                                 "Access-Control-Allow-Origin: *\r\n"
DEBUG:                                 "Access-Control-Allow-Credentials: true\r\n"
DEBUG:                                 "Via: 1.1 vegur\r\n"
DEBUG:                                 "\r\n"
DEBUG:                      

In [5]:
r = HTTP.request("GET", "https://api.github.com/search/repositories?q=more+useful+keyboard"; verbose=3)
println(r.status)
println(String(r.body))

# Need to provide a user agent via eg https://juliaweb.github.io/HTTP.jl/stable/index.html#HTTP.MessageRequest.setuseragent! as per https://developer.github.com/v3/#user-agent-required

DEBUG: 2018-11-29T14:04:27.719 436f ➡️  "GET /search/repositories?q=more+useful+keyboard HTTP/1.1\r\n" (write)
DEBUG: 2018-11-29T14:04:27.737 436f ➡️  "Host: api.github.com\r\n" (write)
DEBUG: 2018-11-29T14:04:27.737 436f ➡️  "Content-Length: 0\r\n" (write)
DEBUG: 2018-11-29T14:04:27.738 436f ➡️  "\r\n" (write)
DEBUG: 2018-11-29T14:04:27.92  436f ⬅️  "HTTP/1.0 403 Forbidden\n" (readavailable)
DEBUG:                                 "Cache-Control: no-cache\n"
DEBUG:                                 "Connection: close\n"
DEBUG:                                 "Content-Type: text/html\n"
DEBUG:                                 "\n"
DEBUG:                                 "Request forbidden by administrative rules. Please make sure your request has a User-Agent header (http://developer.github.com/v3/#user-agent-required). Check https://developer.github.com for other possible causes.\n"
DEBUG: 2018-11-29T14:04:27.92  436f ♻️  "Request forbidden by administrative rules. Please make sure your re

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(403, HTTP.Messages.Response:
"""
HTTP/1.0 403 Forbidden
Cache-Control: no-cache
Connection: close
Content-Type: text/html

Request forbidden by administrative rules. Please make sure your request has a User-Agent header (http://developer.github.com/v3/#user-agent-required). Check https://developer.github.com for other possible causes.
""")